<a href="https://colab.research.google.com/github/ohseungtae/Project_ORI-Team_drive_insurance/blob/main/EDA/OST_EDA_%EC%9A%B4%EC%A0%84%EB%B3%B4%ED%97%98_%EA%B5%90%EC%B0%A8%ED%8C%90%EB%A7%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/kaggle/자동차 보험 반응 분류대회/data
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/kaggle/자동차 보험 반응 분류대회/data')
print(sys.path)

/content/drive/MyDrive/Colab Notebooks/kaggle/자동차 보험 반응 분류대회/data
['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Colab Notebooks/kaggle/자동차 보험 반응 분류대회/data']


In [ ]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [ ]:
train2= pd.read_csv('train2.csv')

In [ ]:
train= pd.concat([train, train2], ignore_index=True)

In [ ]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

In [ ]:
train["car_sum"] = train["Vehicle_Damage"].astype("str") + train["Driving_License"].astype("str") + train["Previously_Insured"].astype("str")
train["car_sum"] = train["car_sum"].astype("category").cat.codes

test["car_sum"] = test["Vehicle_Damage"].astype("str") + test["Driving_License"].astype("str") + test["Previously_Insured"].astype("str")
test["car_sum"] = test["car_sum"].astype("category").cat.codes

In [ ]:
train['Gender'] = train['Gender'].replace({'Male': 0, 'Female': 1})
test['Gender'] = test['Gender'].replace({'Male': 0, 'Female': 1})

train['Vehicle_Age'] = train['Vehicle_Age'].replace({'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2})
test['Vehicle_Age'] = test['Vehicle_Age'].replace({'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2})

train['Vehicle_Damage'] = train['Vehicle_Damage'].replace({'Yes': 1, 'No': 0})
test['Vehicle_Damage'] = test['Vehicle_Damage'].replace({'Yes': 1, 'No': 0})

In [ ]:
from sklearn.preprocessing import MinMaxScaler

selected_columns = ['Annual_Premium', 'Policy_Sales_Channel', 'Vintage']
minMax = MinMaxScaler()

# 훈련 데이터와 테스트 데이터 스케일링
train[selected_columns] = minMax.fit_transform(train[selected_columns])
test[selected_columns] = minMax.transform(test[selected_columns])

In [ ]:
from sklearn.model_selection import train_test_split
X = train.drop('Response', axis=1)
y = train['Response']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#X_tr, X_val, y_tr, y_val = train_test_split 다른 사람은 이거로 한 번 더 데이터를 나눔

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import xgboost as xgb
from sklearn.metrics import roc_auc_score
# 하이퍼파라미터의 검색 공간 정의
space = {
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    #'n_estimators': hp.quniform('n_estimators', 100, 500, 50),
    'gamma': hp.uniform('gamma', 0, 0.5),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1)
}

# 목적 함수 정의
def objective(params):
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = 100

    model = xgb.XGBClassifier(**params, use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)
    preds = model.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, preds)

    return {'loss': -roc_auc, 'status': STATUS_OK}

# Trials 객체 생성
trials = Trials()

# Hyperopt로 최적의 하이퍼파라미터 탐색
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=30,
            trials=trials)

print("Best parameters found: ", best)

# Trials 객체 생성
trials = Trials()

100%|██████████| 30/30 [1:33:03<00:00, 186.11s/trial, best loss: -0.8788716581307815]
Best parameters found:  {'colsample_bytree': 0.61047475224101, 'gamma': 0.2728782098188464, 'learning_rate': 0.19392589558523762, 'max_depth': 10.0, 'min_child_weight': 3.0, 'subsample': 0.6733218184101404}


In [ ]:
# 최적의 하이퍼파라미터로 모델 학습 및 평가
best_params = {
    'max_depth': int(best['max_depth']),
    'learning_rate': best['learning_rate'],
    'n_estimators': 300,
    'gamma': best['gamma'],
    'subsample': best['subsample'],
    'colsample_bytree': best['colsample_bytree'],
    'min_child_weight': best['min_child_weight']
}

model = xgb.XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)
preds = model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, preds)

In [ ]:
## 1) 검색 공간 설정

from hyperopt import hp

## LightGBM이 XBGoost에 비해 깊이가 깊음
search_space = {
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'n_estimators': hp.quniform('n_estimators', 100, 500, 50),
    'gamma': hp.uniform('gamma', 0, 0.5),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1)
}

## 2) 목적 함수 설정

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

def objective_func(search_space):

  xgb_clf = XGBClassifier(
      n_estimators=100,
      max_depth=int(search_space['max_depth']),                ## int형으로 형변환 필요
      min_child_weight=int(search_space['min_child_weight']),  ## int형으로 형변환 필요
      learning_rate=search_space['learning_rate'],
      subsample=search_space['subsample'],
      num_leaves=int(search_space['num_leaves']),
      #eval_metric='logloss' => 불균형 데이터셋이므로 성능 평가 지표를 roc-auc로 설정
      )

      ## XGBoost와 LightGBM에서는 cross_val_score()를 적용하면 early stopping 지원 불가, KFold 방식으로 직접 구현해야 함
      ## 3개의 k-fold 방식으로 평가된 roc-auc 지표를 담는 list
  roc_auc_list = []

  ## 3개의 k-fold 방식 적용
  kf = KFold(n_splits=3)

  for tr_index, val_index in kf.split(X_train):

    X_tr, y_tr = X_train.iloc[tr_index], y_train.iloc[tr_index]
    X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]

    xgb_clf.fit(
        X_tr, y_tr,
        eval_metric='auc',
        eval_set=[(X_tr, y_tr), (X_val, y_val)]
    )

    score = roc_auc_score(
        y_val,
        xgb_clf.predict_proba(X_val)[:, 1]
    )

    roc_auc_list.append(score)

  return (-1) * np.mean(roc_auc_list)


In [ ]:
# 3) fmin()을 사용하여 최적 하이퍼 파라미터 찾기

from hyperopt import fmin, tpe, Trials

trials = Trials()

best = fmin(
    fn=objective_func,
    space=lgbm_search_space,
    algo=tpe.suggest,
    max_evals=30,  ## 30번 만큼 반복하며 최적의 하이퍼 파라미터 찾음
    trials=trials,
)

In [ ]:
## 획득한 최적의 하이퍼 파라미터를 이용하여 모델 선언
lgbm_wrapper = LGBMClassifier(
    n_estimators=500,
    num_leaves=int(best['num_leaves']),
    learning_rate=round(best['learning_rate'], 5),
    max_depth=int(best['max_depth']),
    min_child_weight=int(best['min_child_weight']),
    subsample=round(best['subsample'], 5)
)

In [ ]:
lgbm_wrapper.fit(
    X_tr, y_tr,
    eval_metric='auc',
    eval_set=[(X_tr, y_tr), (X_test, y_test)]
)

In [ ]:
lgbm_roc_score = roc_auc_score(
    y_test,
    lgbm_wrapper.predict_proba(X_test)[:, 1]
)

lgbm_roc_score

In [ ]:
#피처중요도
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from lightgbm import plot_importance


ax = lgb.plot_importance(lgbm_clf, importance_type='split', max_num_features=10, figsize=(10, 6))
plt.title('Feature Importance (Split)')
plt.show()